#### Jump distance cutoff for maximum percolation dimensionality

In [32]:
from ase.io import read
from ions import Percolator

file = '/Users/artemdembitskiy/Downloads/LiFePO4.cif'
atoms = read(file)  

pl = Percolator(
                atoms, 
                mobile_specie='Li',
                upper_bound=5.0,   # upper bound for Li-Li hops search, the recommened value is 8.0
                )

bottleneck_radius = 0.5 # Minimum allowed distance between the Li-Li edge and the framework
mincut, maxdim = pl.mincut_maxdim(bottleneck_radius)

print(f'Maximum percolation dimensionality: {maxdim}')
print(f'Jump distance cutoff: {round(mincut, 2)} angstrom', '\n')

Maximum percolation dimensionality: 2
Jump distance cutoff: 4.76 angstrom 



#### Percolation radii for a given cutoff

In [33]:
for dim in [1, 2, 3]:
    percolation_radius = pl.percolation_threshold(dim, cutoff = mincut)
    print(f'{dim}D percolation radius: {round(percolation_radius, 2)} angstrom')

1D percolation radius: 1.58 angstrom
2D percolation radius: 1.46 angstrom
3D percolation radius: 0 angstrom


#### Inequivalent ionic jumps (edges) forming a percolating network

In [30]:
from ase.io import read
from ions import Percolator

file = '/Users/artemdembitskiy/Downloads/LiFePO4.cif'
atoms = read(file)  

pl = Percolator(atoms, mobile_specie='Li', upper_bound=5.0)

bottleneck_radius = 0.5 
mincut, maxdim = pl.mincut_maxdim(bottleneck_radius)
print(f'Maximum percolation dimensionality: {maxdim}')
print(f'Jump distance cutoff: {mincut} angstrom', '\n')

edges = pl.unique_edges(mincut, bottleneck_radius, method = 'naive')
print(f'Inequivalent edges forming {maxdim}D percolating network:')
print(edges)

Maximum percolation dimensionality: 2
Jump distance cutoff: 4.755859375 angstrom 

Inequivalent edges forming 2D percolating network:
[[ 0  3  0 -1  0]
 [ 0  0 -1  0  0]]


- Each raw in the edges array corresponds to the symmetrically unique ionic jump.

- Each edge is represented by its source and target indices in the structure and lattice offset.
- The offset tells you whether it is a jump within a unit cell or a jump through the periodic boundary along some direction.

In [4]:
edge = edges[0]
edge

array([ 0,  3,  0, -1,  0])

In [5]:
source, target, offset = edge[0], edge[1], edge[2:]

#### Initial guess vacancy migration trajectories for nudged elastic band calculations

In [6]:
from ions import Edge

trajectories = []
for edge in edges:
    source, target, offset = edge[0], edge[1], edge[2:]
    
    # create a helper Edge object
    edge = Edge(atoms, source, target, offset)
    print(edge)
    
    # create an edge in a supercell
    superedge = edge.superedge(r_cut = 7.0)
    
    # interpolate trajectory
    images = superedge.interpolate(n_images = 7)
    
    # collect trajectories
    trajectories.append(images)

# do stuff

Edge(source=0, target=3, offset=(0, -1, 0), length=3.05)
Edge(source=0, target=0, offset=(-1, 0, 0), length=4.75)


#### Perfoming NEB calculation using universal machine learning interatomic potentials

In [ ]:
from ase.mep import NEB
from ase.optimize import FIRE
from sevenn.calculator import SevenNetCalculator

# preload calculators

calculators = [SevenNetCalculator(model='7net-0', device ='cpu') for _ in range(len(trajectories[0]))]

for traj in trajectories:
    for image, calc in zip(traj, calculators):
        image.calc = calc
    
    # optimize source
    optim = FIRE(traj[0], logfile = None)
    optim.run(fmax=0.01, steps = 100)
    
    # optimize target
    optim = FIRE(traj[-1], logfile = None)
    optim.run(fmax=0.01, steps = 100)

    neb = NEB(traj, k = 2.0)

    # optimize string
    optim = FIRE(neb, logfile=None)
    optim.run(fmax=0.01, steps = 100)

    # ideally you should check for NEB forces convergence
    # we skip this part

#### Percolation barriers from UMLIP-NEB calculations

In [8]:
emins = []
emaxs = []
for traj in trajectories:
    profile = [image.get_potential_energy() for image in traj]
    emins.append(min(profile))
    emaxs.append(max(profile))

barriers = pl.find_percolation_barriers(mincut, bottleneck_radius, emins, emaxs, method='naive')
barriers

/Users/artemdembitskiy/opt/anaconda3/envs/cgcnn/lib/python3.10/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['equivalent_atoms']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(


{'e1d': 0.2144775390625, 'e2d': 1.83258056640625, 'e3d': None}

#### Bond valence site energy (BVSE) NEB calculations of percolation barriers

In [9]:
from ase.io import read
from ions import Percolator
from ions.decorate import assign_bvse_params, assign_oxidation_states

file = '/Users/artemdembitskiy/Downloads/LiFePO4.cif'
atoms = read(file)  

assign_oxidation_states(atoms)
assign_bvse_params(atoms, 'Li', 1, self_interaction=True)

pl = Percolator(
                atoms, 
                mobile_specie='Li',
                upper_bound=5.0,   # upper bound for Li-Li hops search 
                )

bottleneck_radius = 0.5 # Minimum allowed distance between the Li-Li edge and the framework
mincut, maxdim = pl.mincut_maxdim(bottleneck_radius)

print(f'Maximum percolation dimensionality: {maxdim}')
print(f'Jump distance cutoff: {round(mincut, 2)} angstrom', '\n')

Maximum percolation dimensionality: 2
Jump distance cutoff: 4.76 angstrom 



In [10]:
from ions import Edge

trajectories = []
for edge in edges:
    source, target, offset = edge[0], edge[1], edge[2:]
    
    # create a helper Edge object
    edge = Edge(atoms, source, target, offset)
    print(edge)
    
    # create an edge in a supercell
    superedge = edge.superedge(r_cut = 8.0)
    
    # interpolate trajectory
    images = superedge.interpolate(n_images = 7)
    
    # collect trajectories
    trajectories.append(images)

# do stuff

Edge(source=0, target=3, offset=(0, -1, 0), length=3.05)
Edge(source=0, target=0, offset=(-1, 0, 0), length=4.75)


In [11]:
import numpy as np
from ase.mep import NEB
from ase.optimize import FIRE
from ions.calculators.bvse_calculator import BVSECalculator

fmax = 0.1
steps = 100
for traj in trajectories:
    for image in traj:
        sites = np.argwhere(image.arrays['moving'] == True).ravel()
        image.calc = BVSECalculator(sites = sites)

    
    optim = FIRE(traj[0], logfile = None)
    optim.run(fmax = fmax, steps = steps)

    optim = FIRE(traj[-1], logfile = None)
    optim.run(fmax = fmax, steps = steps)

    neb = NEB(traj, k=2.0)
    optim = FIRE(neb, logfile = None)
    optim.run(fmax = fmax, steps = steps)


In [12]:
emins = []
emaxs = []
for traj in trajectories:
    profile = [image.get_potential_energy() for image in traj]
    emins.append(min(profile))
    emaxs.append(max(profile))

barriers = pl.find_percolation_barriers(mincut, bottleneck_radius, emins, emaxs, method='naive')
barriers

/Users/artemdembitskiy/opt/anaconda3/envs/cgcnn/lib/python3.10/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['equivalent_atoms']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(


{'e1d': 0.35584832545497247, 'e2d': 3.3530108951333237, 'e3d': None}